In [2]:
import pandas as pd
import neattext.functions as nfx

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [4]:
df = pd.read_csv('2021-10-19-MichiganOnline-courses.csv')

In [5]:
df

,name,description,experience_type
0,Accounting for Decision Making,"Through this course, you will start by address...",course
1,Achieving Health Equity in Healthcare,The third and the last course of the Addressin...,course
2,"Act on Climate: Steps to Individual, Community...",Are you concerned about climate change? Would ...,course
3,Additive Manufacturing,"This course, Additive Manufacturing, is the th...",course
4,Advanced Concepts in Time Value of Money (TVM),This course builds upon the fundamental concep...,course
...,...,...,...
516,Women & Incarceration (Seeking Justice: Conver...,This module examines the impacts of incarcerat...,course
517,Writing and Editing: Drafting,This third course in the “Good with Words: Wri...,course
518,Writing and Editing: Revising,This fourth and final course in the “Good with...,course
519,Writing and Editing: Structure and Organization,This second course in the Good with Words: Wri...,course


In [6]:
df['description']

0      Through this course, you will start by address...
1      The third and the last course of the Addressin...
2      Are you concerned about climate change? Would ...
3      This course, Additive Manufacturing, is the th...
4      This course builds upon the fundamental concep...
                             ...                        
516    This module examines the impacts of incarcerat...
517    This third course in the “Good with Words: Wri...
518    This fourth and final course in the “Good with...
519    This second course in the Good with Words: Wri...
520    This course will teach you how to use your wri...
Name: description, Length: 521, dtype: object

In [8]:
df['clean_description'] = df['description'].apply(nfx.remove_stopwords)

In [9]:
df['clean_description'] = df['description'].apply(nfx.remove_special_characters)

In [10]:
df[['description', 'clean_description']]

,description,clean_description
0,"Through this course, you will start by address...",Through this course you will start by addressi...
1,The third and the last course of the Addressin...,The third and the last course of the Addressin...
2,Are you concerned about climate change? Would ...,Are you concerned about climate change Would y...
3,"This course, Additive Manufacturing, is the th...",This course Additive Manufacturing is the thir...
4,This course builds upon the fundamental concep...,This course builds upon the fundamental concep...
...,...,...
516,This module examines the impacts of incarcerat...,This module examines the impacts of incarcerat...
517,This third course in the “Good with Words: Wri...,This third course in the Good with Words Writi...
518,This fourth and final course in the “Good with...,This fourth and final course in the Good with ...
519,This second course in the Good with Words: Wri...,This second course in the Good with Words Writ...


In [11]:
# Vectorize description text
count_vect = CountVectorizer()
cv_mat = count_vect.fit_transform(df['clean_description'])

## Explore Semester data

In [5]:
pd.read_excel('LSE requirements/CE Courses Offered FA21.xlsx', skiprows=1)

,Course ID,Offer Nbr,Term,Session,Section,Crse Attr,CrsAtr Val,Term.1,Subject,Catalog,Descr
0,6578,1,2360,1,3,CE,CE,2360,GUITAR,111,Intro Guitar
1,50218,1,2360,1,1,CE,CE,2360,DIGITAL,355,Topics Dig Creat Exp
2,50064,2,2360,1,1,CE,CE,2360,DIGITAL,394,Topics in Dig Med Pr
3,50218,1,2360,1,2,CE,CE,2360,DIGITAL,355,Topics Dig Creat Exp
4,39519,1,2360,1,11,CE,CE,2360,ARTDES,120,Studio: 3D
...,...,...,...,...,...,...,...,...,...,...,...
1481,20088,1,2360,1,279,CE,CE,2360,JAZZ,101,Jazz Perform
1482,20089,1,2360,1,279,CE,CE,2360,JAZZ,102,Jazz Perform
1483,20090,1,2360,1,279,CE,CE,2360,JAZZ,201,Jazz Perform
1484,2821,1,2360,1,1,CE,CE,2360,VDAGAMBA,100,Performance


In [6]:
pd.read_excel('LSE requirements/HU Courses Offered FA21.xlsx', skiprows=1)

,Course ID,Offer Nbr,Term,Session,Section,Crse Attr,CrsAtr Val,Term.1,Subject,Catalog,Descr
0,47420,1,2360,GB2,1,HU,HU,2360,ALA,160,HU Topics in ALA
1,11312,2,2360,1,2,HU,HU,2360,ENGLISH,379,Lit Afam Cult
2,50441,1,2360,1,1,HU,HU,2360,AAS,393,Race Gend Pop Cult
3,8529,1,2360,1,1,HU,HU,2360,HISTART,393,Undergrad Seminar
4,48974,2,2360,1,1,HU,HU,2360,ENGLISH,355,Southern Natures
...,...,...,...,...,...,...,...,...,...,...,...
1389,42269,1,2360,1,3,HU,HU,2360,AMCULT,215,Intro Arab-Amer Std
1390,5537,1,2360,1,1,HU,HU,2360,HISTART,250,Ital Renaissance Art
1391,5537,2,2360,1,1,HU,HU,2360,MEMS,250,Ital Renaissance Art
1392,5537,1,2360,1,2,HU,HU,2360,HISTART,250,Ital Renaissance Art


In [7]:
pd.read_excel('LSE requirements/ID Courses Offered FA21.xlsx', skiprows=1)

,Course ID,Offer Nbr,Term,Session,Section,Crse Attr,CrsAtr Val,Term.1,Subject,Catalog,Descr
0,18533,1,2360,1,1,ID,ID,2360,AMCULT,342,Hist of Family in US
1,18533,1,2360,1,2,ID,ID,2360,AMCULT,342,Hist of Family in US
2,18533,1,2360,1,3,ID,ID,2360,AMCULT,342,Hist of Family in US
3,18533,1,2360,1,4,ID,ID,2360,AMCULT,342,Hist of Family in US
4,20268,3,2360,1,1,ID,ID,2360,ENVIRON,270,Globaliztn & Environ
...,...,...,...,...,...,...,...,...,...,...,...
215,45612,1,2360,1,14,ID,ID,2360,COGSCI,200,Intro to Cog Sci
216,45612,1,2360,1,15,ID,ID,2360,COGSCI,200,Intro to Cog Sci
217,45612,1,2360,1,100,ID,ID,2360,COGSCI,200,Intro to Cog Sci
218,47459,1,2360,1,1,ID,ID,2360,ENVIRON,155,Interdis Env Topics
